<a href="https://colab.research.google.com/github/elisaboccini/AMDproject/blob/main/V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Market Basket Analysis**

Kaggle API

In [1]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elisaboccini","key":"3024605213579b61d22c8ab4775d25f4"}'}

In [2]:
#kaggle API 
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -q ashirwadsangwan/imdb-dataset
!unzip imdb-dataset.zip

kaggle.json
Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


Spark Session

In [3]:
#Spark session creation

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 60kB/s 
     |████████████████████████████████| 204kB 48.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5b0a500cc4de383a196abbf1598dfdbf9f449fb8c2ee02ab2b06595f41522742
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init('spark-3.1.1-bin-hadoop2.7')

In [6]:
from pyspark.sql import SparkSession

In [7]:
#spark = SparkSession.builder.master("local[*]").getOrCreate()
#try with 4 g to avoid garbage collection
spark = SparkSession.builder.master("local")\
        .appName("MBA")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.memory", "15g") \
        .config("spark.driver.maxResultSize", "1g") \
        .config("spark.sql.broadcastTimeout", "10h") \
        .getOrCreate()

In [8]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [9]:
#sc = spark.sparkContext

Dataset

In [10]:
principals = spark.read.csv('title.principals.tsv.gz', sep="\t", header=True, inferSchema=True)
#akas = spark.read.csv('title.akas.tsv.gz', sep="\t", header=True, inferSchema=True)
titlebasics = spark.read.csv('title.basics.tsv.gz', sep="\t", header=True, inferSchema=True)
#ratings = spark.read.csv('title.ratings.tsv.gz', sep="\t", header=True, inferSchema=True)
namesbasics = spark.read.csv('name.basics.tsv.gz', sep="\t", header=True, inferSchema=True)

In [11]:
principals.createOrReplaceTempView("principals")
#akas.createOrReplaceTempView("akas")
titlebasics.createOrReplaceTempView("titlebasics")
#ratings.createOrReplaceTempView("ratings")
namesbasics.createOrReplaceTempView("namesbasics")

In [12]:
# drop duplicates
titlebasics.dropDuplicates((['tconst'])).count() # 6326545
namesbasics.dropDuplicates((['nconst'])).count() #9711022

9711022

In [13]:
#which are the types of items collected in the titlebasics db?
spark.sql('''SELECT distinct (titleType)
              FROM titlebasics''').show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



In [14]:
#Taking only actors (not crew) and only movies and shorts (not tvSeries, tvMiniSeries, tvEpisode, tvSpecial,video, videogames)
data = spark.sql('''SELECT tb.tconst, tb.primaryTitle, nb.primaryName, nb.nconst 
                          FROM titlebasics as tb LEFT JOIN principals as p ON tb.tconst = p.tconst INNER JOIN namesbasics as nb ON p.nconst = nb.nconst
                          WHERE ((p.category = 'actor' OR p.category = 'actress') AND (tb.titleType = 'movie' OR tb.titleType = 'tvMovie' OR tb.titleType = 'short' OR tb.titleType = 'tvShort')) ''')

data.createOrReplaceTempView('data')
data.show(5)
data.count() #3686467-> this also reduces a lot the size of the db (from 14 mln to 3 mln)

+---------+-----------------+--------------+---------+
|   tconst|     primaryTitle|   primaryName|   nconst|
+---------+-----------------+--------------+---------+
|tt0083109| The Cabbage Soup|Louis de Funès|nm0000086|
|tt0062120|The Little Bather|Louis de Funès|nm0000086|
|tt0057422|    Squeak-squeak|Louis de Funès|nm0000086|
|tt0043691|     The Gamblers|Louis de Funès|nm0000086|
|tt0335439| Station mondaine|Louis de Funès|nm0000086|
+---------+-----------------+--------------+---------+
only showing top 5 rows



3686467

In [15]:
spark.sql('''select count(distinct tconst) 
            from data''').show() # 974512 -> we have actually 1/3 of the unique movies we had before!

+----------------------+
|count(DISTINCT tconst)|
+----------------------+
|                974512|
+----------------------+



In [16]:
#lets see if we can group by primarytitle (if the distinct titles are equivalent to n of tconst)
spark.sql('''select count(distinct primaryTitle) 
            from data''').show() #756111 -> there are homonyms in movies and shorts

+----------------------------+
|count(DISTINCT primaryTitle)|
+----------------------------+
|                      756111|
+----------------------------+



FP-Growth: pre-processing

In [17]:
#baskets creation
from pyspark.sql.functions import collect_set

moviesasbaskets = data.groupBy('tconst','primaryTitle').agg(collect_set('primaryName').alias('cast')) #metti un alias
moviesasbaskets.createOrReplaceTempView('moviesasbaskets')

moviesasbaskets.show(n=5) #974512 -> correct! we have a basket for each unique tconst

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|                cast|
+---------+--------------------+--------------------+
|tt0000007|Corbett and Court...|[James J. Corbett...|
|tt0000378|          Les vagues|[Julie Souplet, G...|
|tt0000621|   That Fatal Sneeze|[Gertie Potter, T...|
|tt0000653|A Calamitous Elop...|[John R. Cumpson,...|
|tt0000841|       The Day After|[Marion Leonard, ...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [18]:
#concatenate using hyphen
# to have info about the title but still maintaining the basket form (so just one col for the basket and the corresponding
# list of itemsets)
from pyspark.sql import functions as f

moviesasbaskets = moviesasbaskets.withColumn('movie', f.concat(f.col('tconst'),f.lit('-'),f.col('primaryTitle')))

moviesasbaskets = moviesasbaskets.select('movie', 'cast')
moviesasbaskets.show(5)

+--------------------+--------------------+
|               movie|                cast|
+--------------------+--------------------+
|tt0000007-Corbett...|[James J. Corbett...|
|tt0000378-Les vagues|[Julie Souplet, G...|
|tt0000621-That Fa...|[Gertie Potter, T...|
|tt0000653-A Calam...|[John R. Cumpson,...|
|tt0000841-The Day...|[Marion Leonard, ...|
+--------------------+--------------------+
only showing top 5 rows



In [19]:
# Caching aka persisting-> otherwise randomsplit will lead to inconsistencies, since it changes the randomsplit each
# time you perform an operation on the splitted data
moviesasbaskets = moviesasbaskets.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
# apply the randomSplit() 
MBAtrain,MBAtest = moviesasbaskets.randomSplit([0.9, 0.1], seed=13)

FP-Growth: Model implementation

In [34]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="cast", minSupport=0.0003, minConfidence=0)
modelFP = fpGrowth.fit(MBAtrain)  
#excluding the episodes we consider frequent an actor if it appears in (distinct tconst = 974512)*0.0001 97 films 
#reasonable

FP-Growth: Results

In [35]:
# Display frequent itemsets.-> NB this result includes all size items!
from pyspark.sql.functions import col, asc,desc
freqItemsets = modelFP.freqItemsets
freqItemsets.createOrReplaceTempView("freqItemsets")
freqItemsets.orderBy(col('freq').desc()).show()
#the most frequent actor is no more nm10120013 appearing in > 5000 films BUT Mel Blanc with 925 movies
#however taking as minSupport 0.0005 only 5 actors are retrieved and therefore no association rules
#therefore try again with a minSupport of 0.0001-> now it works
#try also to set minSUpport as 0.0003 as in the Apriori analysis

+--------------------+----+
|               items|freq|
+--------------------+----+
|         [Mel Blanc]| 925|
|  [Matsunosuke Onoe]| 837|
|      [Brahmanandam]| 725|
|       [Adoor Bhasi]| 529|
|      [Eddie Garcia]| 475|
|       [Jack Mercer]| 406|
|         [Lee Moran]| 393|
|        [Prem Nazir]| 384|
|   [Jack Richardson]| 384|
|       [Eddie Lyons]| 378|
|      [Sung-il Shin]| 370|
|      [Paquito Diaz]| 356|
|      [Oliver Hardy]| 355|
|  [Masayoshi Nogami]| 352|
|[J. Warren Kerrigan]| 339|
|         [Mammootty]| 339|
|          [Mohanlal]| 314|
|    [Aachi Manorama]| 313|
|       [Mae Questel]| 309|
|           [Bahadur]| 308|
+--------------------+----+
only showing top 20 rows



In [36]:
# Display generated association rules.
associationRules = modelFP.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show(n=5) 

+-------------+-------------+------------------+------------------+--------------------+
|   antecedent|   consequent|        confidence|              lift|             support|
+-------------+-------------+------------------+------------------+--------------------+
|[Eddie Lyons]|  [Lee Moran]| 0.783068783068783|1747.4588365173606|3.375142531356898...|
|  [Lee Moran]|[Eddie Lyons]|0.7531806615776081|1747.4588365173606|3.375142531356898...|
+-------------+-------------+------------------+------------------+--------------------+



In [37]:
#now only two rows in association rules having confidence equal to 1
spark.sql("""select (*)
           from associationRules
           where confidence = '1.0' 
           limit 20""").show()
#We obtain that where there are Curly Howard and Larry Fine there is Moe Howard and where there are Curly Howard and Moe Howard there is Larry Fine

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



In [38]:
#for a better understanding ###INTERESTING
spark.sql("""select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
              from associationRules
              order by confidence desc
              limit 20""").show()

+---------------+-----------------+------------------+
|antecedent (if)|consequent (then)|        confidence|
+---------------+-----------------+------------------+
|  [Eddie Lyons]|      [Lee Moran]| 0.783068783068783|
|    [Lee Moran]|    [Eddie Lyons]|0.7531806615776081|
+---------------+-----------------+------------------+



In [39]:
# transform examines the input items against all the association rules and summarize the consequents as prediction
# it would be useful to divide the db in training and test set in order to actually do predictions
association = modelFP.transform(MBAtest)
association.show()

+--------------------+--------------------+----------+
|               movie|                cast|prediction|
+--------------------+--------------------+----------+
|tt0000378-Les vagues|[Julie Souplet, G...|        []|
|   tt0006235-The Vow|    [Marion Leonard]|        []|
|tt0006399-The Ban...|[Francis Ford, Jo...|        []|
|tt0006538-The Com...|[Harold Lockwood,...|        []|
|tt0007213-The Pri...|[Frank Whitson, D...|        []|
|tt0017047-The Win...|[Ivan Klyukvin, V...|        []|
|tt0017055-The Lad...|[Roy Stewart, Ral...|        []|
|tt0018691-Beware ...|[William Collier ...|        []|
|tt0020073-The Lad...|[Mona Goya, Bruce...|        []|
|tt0027147-Two for...|[Mary Boland, Bin...|        []|
|tt0028509-Windbag...|[Norma Varden, Gr...|        []|
|tt0029282-My Arti...|[Mae Questel, Jac...|        []|
|tt0030958-The Nov...|[Jean Galland, Je...|        []|
| tt0034335-Under Age|[Mary Anderson, T...|        []|
|tt0035213-Le prin...|[Jimmy Gaillard, ...|        []|
|tt0035788

In [40]:
#Doing predictions for MBAtest2
association.createOrReplaceTempView('association')
predFP= spark.sql("""select *
                      from association
                      where size(prediction) >= 1 """)
predFP.createOrReplaceTempView('predFP')
predFP.show() #718 rows in pred_ex2

+--------------------+--------------------+-------------+
|               movie|                cast|   prediction|
+--------------------+--------------------+-------------+
|tt0256990-The Par...|[William Bertram,...|  [Lee Moran]|
|tt1318984-Three o...|[Dick Dewell, Jac...|[Eddie Lyons]|
|tt0249766-A Lucky...|       [Eddie Lyons]|  [Lee Moran]|
|tt0332698-A Man o...|[Stella Adams, Do...|[Eddie Lyons]|
|tt0334724-Women F...|[Alberta Vaughn, ...|[Eddie Lyons]|
|tt0345092-Cupid's...|[Stella Adams, Ed...|  [Lee Moran]|
|    tt0016415-Tessie|[May McAvoy, Myrt...|[Eddie Lyons]|
|tt0256893-Love, W...|[William Bertram,...|  [Lee Moran]|
| tt0024482-Racetrack|[Leo Carrillo, Ka...|[Eddie Lyons]|
|tt0334696-Weighed...|[Lee Moran, Ramon...|[Eddie Lyons]|
|tt1334291-When He...|       [Eddie Lyons]|  [Lee Moran]|
|tt0862778-Only a ...|       [Eddie Lyons]|  [Lee Moran]|
|tt0017446-Syncopa...|[Rockliffe Fellow...|[Eddie Lyons]|
|tt0014829-Daring ...|[Norman Kerry, Be...|[Eddie Lyons]|
|tt0830680-Tha

In [41]:
#focus the analysis for frequent itemset with one actor only (frequent actors)
#NOW filter freqItemsets by those having items = 1 -> most frequent actors banally
mostfrequentactor= spark.sql("""select items, freq
                                    from freqItemsets
                                    where size(items) = 1
                                    order by freq desc""")
mostfrequentactor.show(n=5)
mostfrequentactor.count()

+------------------+----+
|             items|freq|
+------------------+----+
|       [Mel Blanc]| 925|
|[Matsunosuke Onoe]| 837|
|    [Brahmanandam]| 725|
|     [Adoor Bhasi]| 529|
|    [Eddie Garcia]| 475|
+------------------+----+
only showing top 5 rows



41

In [42]:
#now analysis for frequent itemset with two actors
mostfrequent2actors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) = 2
                                      order by freq desc""")
mostfrequent2actors.show(n=5)
mostfrequent2actors.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Eddie Lyons, Lee...| 296|
+--------------------+----+



1

In [43]:
#now analysis for frequent itemset with three actors
mostfrequent3actors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) = 3
                                      order by freq desc""")
mostfrequent3actors.show(n=5)
mostfrequent3actors.count() 

+-----+----+
|items|freq|
+-----+----+
+-----+----+



0

In [31]:
#for more than 3 actors there is only an itemset, containing 4 actors, considered frequent according to our parameter and having freq = 88
#now analysis for frequent itemset with three actors
mostfrequentnactors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) > 3
                                      order by freq desc""")
mostfrequentnactors.show(n=5)
mostfrequentnactors.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Sammy Brooks, Ha...|  88|
+--------------------+----+



1

In [33]:
mostfrequentnactors.toPandas()['items'].tolist() 
#just to visualize that the 4 most frequent actors are [['Sammy Brooks', 'Harold Lloyd', 'Bebe Daniels', "'Snub' Pollard"]]
#appearing together in 88 films

[['Sammy Brooks', 'Harold Lloyd', 'Bebe Daniels', "'Snub' Pollard"]]